# Chapter 19: Advanced Ray Tracing Topics

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adiel2012/computer-vision/blob/main/chapter_19_advanced_ray_tracing_topics.ipynb)

**Modern ray tracing** encompasses real-time rendering, denoising, hybrid methods, and cutting-edge research. This chapter covers techniques enabling real-time ray tracing in games and interactive applications.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random
from typing import Tuple, List
from dataclasses import dataclass

## 1. Real-Time Ray Tracing

**Real-time RT** targets 30-60 FPS with limited samples per pixel.

### Challenges

- **Low sample counts**: 1-4 SPP → high noise
- **Tight frame budgets**: 16-33ms per frame
- **Complex scenes**: Millions of triangles
- **Dynamic content**: Moving objects, changing lights

### Strategies

1. **Hybrid rendering**: Rasterization + selective RT
2. **Aggressive denoising**: Temporal + spatial filters
3. **Hardware acceleration**: RTX cores, BVH hardware
4. **Adaptive sampling**: More samples where needed
5. **Temporal reuse**: Accumulate across frames

### Performance Budget (60 FPS)

$$
16.67 \text{ ms/frame} = \text{G-buffer} + \text{RT} + \text{denoise} + \text{compose}
$$

Typical split:
- G-buffer: 3-5 ms
- Ray tracing: 5-8 ms  
- Denoising: 3-5 ms
- Composition: 1-2 ms

In [ ]:
class RealTimeConfig:
    """Configuration for real-time ray tracing"""
    def __init__(self):
        # Sample counts
        self.shadow_rays_per_pixel = 1  # Hard shadows
        self.reflection_rays = 1  # Single reflection
        self.ao_rays = 1  # Ambient occlusion
        self.gi_rays = 1  # Indirect lighting
        
        # Quality vs performance
        self.max_bounces = 3  # Limited recursion
        self.russian_roulette_depth = 2
        
        # Resolution scaling
        self.rt_resolution_scale = 0.5  # Half-res RT
        
        # Temporal accumulation
        self.temporal_alpha = 0.05  # Blend with history
        self.max_accumulation = 64  # frames

def estimate_frame_time(config, width, height):
    """Estimate rendering time (simplified)"""
    rt_width = int(width * config.rt_resolution_scale)
    rt_height = int(height * config.rt_resolution_scale)
    
    total_rays = rt_width * rt_height * (
        config.shadow_rays_per_pixel +
        config.reflection_rays +
        config.ao_rays +
        config.gi_rays
    )
    
    # Assume 1M rays/ms on RTX GPU
    rays_per_ms = 1_000_000
    rt_time_ms = total_rays / rays_per_ms
    
    # Add denoising (~20% of RT time)
    denoise_time_ms = rt_time_ms * 0.2
    
    # Add G-buffer (~5ms)
    gbuffer_time_ms = 5.0
    
    total_time_ms = gbuffer_time_ms + rt_time_ms + denoise_time_ms
    
    return {
        'total_ms': total_time_ms,
        'fps': 1000.0 / total_time_ms if total_time_ms > 0 else 0,
        'rt_ms': rt_time_ms,
        'denoise_ms': denoise_time_ms,
        'gbuffer_ms': gbuffer_time_ms
    }

# Example
config = RealTimeConfig()
perf = estimate_frame_time(config, 1920, 1080)
print(f"Estimated performance: {perf['fps']:.1f} FPS")
print(f"Frame time breakdown:")
print(f"  G-buffer:  {perf['gbuffer_ms']:.2f} ms")
print(f"  Ray trace: {perf['rt_ms']:.2f} ms")
print(f"  Denoise:   {perf['denoise_ms']:.2f} ms")
print(f"  Total:     {perf['total_ms']:.2f} ms")

## 2. Temporal Accumulation

**Temporal accumulation** blends current frame with history.

### Algorithm

$$
C_t = \alpha C_{\text{current}} + (1 - \alpha) C_{\text{history}}
$$

where $\alpha \in [0, 1]$ controls blend factor.

### Motion Vectors

Track pixel motion between frames:

$$
\mathbf{v} = \mathbf{p}_{\text{current}} - \mathbf{p}_{\text{previous}}
$$

Reproject previous frame:
$$
\mathbf{p}_{\text{history}} = \mathbf{p}_{\text{current}} - \mathbf{v}
$$

### Disocclusion Detection

Detect when history is invalid:
- Depth discontinuity
- Normal change
- Off-screen history
- New geometry

Increase $\alpha$ for disoccluded pixels.

In [ ]:
def temporal_accumulation(current, history, motion_vectors, alpha=0.05):
    """Temporal accumulation with motion vectors"""
    height, width = current.shape[:2]
    result = np.zeros_like(current)
    
    for y in range(height):
        for x in range(width):
            # Get motion vector
            mv = motion_vectors[y, x]
            
            # Reproject to previous frame
            prev_x = x - int(mv[0])
            prev_y = y - int(mv[1])
            
            # Check if history is valid
            if (0 <= prev_x < width and 0 <= prev_y < height):
                # Blend current with history
                hist_color = history[prev_y, prev_x]
                result[y, x] = alpha * current[y, x] + (1 - alpha) * hist_color
            else:
                # No valid history (disocclusion)
                result[y, x] = current[y, x]
    
    return result

# Simulate temporal accumulation
def demonstrate_temporal_accumulation():
    """Show effect of temporal accumulation on noise"""
    size = 256
    num_frames = 30
    
    # Base image (clean)
    base = np.zeros((size, size, 3))
    for y in range(size):
        for x in range(size):
            base[y, x] = [x/size, y/size, 0.5]
    
    # No temporal accumulation (just current noisy frame)
    noisy_single = base + np.random.normal(0, 0.2, base.shape)
    noisy_single = np.clip(noisy_single, 0, 1)
    
    # With temporal accumulation
    accumulated = base.copy()
    for frame in range(num_frames):
        # Add noise (simulating low SPP)
        noisy = base + np.random.normal(0, 0.2, base.shape)
        noisy = np.clip(noisy, 0, 1)
        
        # Accumulate (no motion for simplicity)
        alpha = 1.0 / (frame + 1)  # Decreasing alpha
        accumulated = alpha * noisy + (1 - alpha) * accumulated
    
    return noisy_single, accumulated

print("✓ Temporal accumulation loaded")

## 3. Spatiotemporal Denoising (SVGF)

**SVGF** (Spatiotemporal Variance-Guided Filtering) for real-time.

### Algorithm Steps

1. **Temporal accumulation**: Blend with reprojected history
2. **Variance estimation**: Track sample variance
3. **Edge-stopping à-trous wavelet**: Multi-scale spatial filter
4. **Guided by variance**: Larger kernel where more variance

### Edge-Stopping Function

$$
w(\mathbf{p}, \mathbf{q}) = w_z \cdot w_n \cdot w_l
$$

- $w_z$: depth similarity
- $w_n$: normal similarity
- $w_l$: luminance similarity

### Variance-Guided Kernel

$$
\sigma_{\text{kernel}} = k \cdot \sqrt{\text{var}(\mathbf{p})}
$$

Larger filter for noisier regions.

In [ ]:
def edge_stopping_weight(center_val, neighbor_val, sigma):
    """Compute edge-stopping weight"""
    diff = np.linalg.norm(center_val - neighbor_val)
    return math.exp(-(diff * diff) / (2 * sigma * sigma))

def atrous_wavelet_filter(image, normals=None, depth=None, iteration=0):
    """À-trous wavelet filter (edge-aware)"""
    height, width = image.shape[:2]
    result = np.zeros_like(image)
    
    # Kernel step size (2^iteration)
    step = 2 ** iteration
    
    # 3x3 kernel offsets
    kernel_offsets = [
        (-1, -1), (0, -1), (1, -1),
        (-1,  0), (0,  0), (1,  0),
        (-1,  1), (0,  1), (1,  1)
    ]
    
    # Gaussian kernel weights
    kernel_weights = np.array([
        1, 2, 1,
        2, 4, 2,
        1, 2, 1
    ], dtype=float) / 16.0
    
    for y in range(height):
        for x in range(width):
            center_color = image[y, x]
            
            sum_weight = 0.0
            sum_color = np.zeros(3)
            
            for i, (dx, dy) in enumerate(kernel_offsets):
                nx = x + dx * step
                ny = y + dy * step
                
                # Clamp to bounds
                nx = np.clip(nx, 0, width - 1)
                ny = np.clip(ny, 0, height - 1)
                
                neighbor_color = image[ny, nx]
                
                # Spatial weight (Gaussian)
                w_spatial = kernel_weights[i]
                
                # Color similarity weight
                w_color = edge_stopping_weight(center_color, neighbor_color, 0.2)
                
                # Combined weight
                weight = w_spatial * w_color
                
                sum_weight += weight
                sum_color += weight * neighbor_color
            
            if sum_weight > 0:
                result[y, x] = sum_color / sum_weight
            else:
                result[y, x] = center_color
    
    return result

print("✓ À-trous wavelet filter loaded")

## 4. Reservoir Sampling (ReSTIR)

**ReSTIR** (Reservoir-based Spatiotemporal Importance Resampling) for efficient direct lighting.

### Problem

With thousands of lights, testing all is expensive:

$$
L = \sum_{i=1}^N L_i
$$

### Solution: Weighted Reservoir Sampling

Maintain $M$ samples in reservoir, update with new sample:

$$
w_{\text{sum}} \leftarrow w_{\text{sum}} + w_i
$$
$$
\text{Accept with probability } \frac{w_i}{w_{\text{sum}}}
$$

### Temporal Reuse

Reuse previous frame's reservoirs:
1. Reproject to previous frame
2. Combine reservoirs
3. Validate with visibility

### Spatial Reuse

Share reservoirs with neighbors:
- More effective samples
- Correlation across pixels

In [ ]:
@dataclass
class Reservoir:
    """Weighted reservoir for importance sampling"""
    sample_id: int = -1  # Currently held sample
    weight_sum: float = 0.0  # Sum of weights
    num_samples: int = 0  # M (number seen)
    
    def update(self, new_sample_id: int, weight: float) -> bool:
        """Update reservoir with new sample"""
        self.weight_sum += weight
        self.num_samples += 1
        
        # Accept with probability w / w_sum
        if random.random() < (weight / self.weight_sum):
            self.sample_id = new_sample_id
            return True
        return False
    
    def get_contribution_weight(self) -> float:
        """Get weight for final contribution"""
        if self.num_samples == 0:
            return 0.0
        return self.weight_sum / self.num_samples

def restir_light_sampling(lights, num_candidates=32):
    """ReSTIR light sampling (simplified)"""
    reservoir = Reservoir()
    
    # Sample candidates
    for _ in range(num_candidates):
        # Randomly select light
        light_id = random.randint(0, len(lights) - 1)
        
        # Compute weight (simplified: just intensity)
        weight = lights[light_id]['intensity']
        
        # Update reservoir
        reservoir.update(light_id, weight)
    
    return reservoir

# Example
lights = [{'intensity': random.random() * 10} for _ in range(1000)]
reservoir = restir_light_sampling(lights)
print(f"Selected light {reservoir.sample_id} from {len(lights)} lights")
print(f"Contribution weight: {reservoir.get_contribution_weight():.2f}")

## 5. Hybrid Rendering

**Hybrid rendering** combines rasterization and ray tracing.

### Typical Pipeline

1. **G-buffer pass** (rasterization):
   - Depth
   - Normals
   - Albedo
   - Roughness/Metallic

2. **Ray tracing passes** (selective):
   - Shadows
   - Reflections
   - Ambient occlusion
   - Global illumination (1 bounce)

3. **Denoising**:
   - Separate denoisers per effect
   - Guided by G-buffer

4. **Composition**:
   - Combine all layers
   - Post-processing

### Benefits

- Fast primary visibility (rasterization)
- High-quality secondary effects (RT)
- Scalable quality/performance trade-off

In [ ]:
class HybridRenderer:
    """Hybrid rasterization + ray tracing renderer (conceptual)"""
    
    def render_frame(self, scene, camera, width, height):
        """Render frame with hybrid approach"""
        
        # 1. G-buffer pass (rasterization)
        gbuffer = self.rasterize_gbuffer(scene, camera, width, height)
        # gbuffer contains: depth, normal, albedo, roughness
        
        # 2. Ray tracing passes (selective)
        shadows = self.trace_shadows(gbuffer, scene, samples_per_pixel=1)
        reflections = self.trace_reflections(gbuffer, scene, samples_per_pixel=1)
        ao = self.trace_ambient_occlusion(gbuffer, scene, samples_per_pixel=1)
        gi = self.trace_global_illumination(gbuffer, scene, samples_per_pixel=1)
        
        # 3. Denoise each layer
        shadows_clean = self.denoise(shadows, gbuffer)
        reflections_clean = self.denoise(reflections, gbuffer)
        ao_clean = self.denoise(ao, gbuffer)
        gi_clean = self.denoise(gi, gbuffer)
        
        # 4. Compose final image
        final = self.compose(
            gbuffer,
            shadows_clean,
            reflections_clean,
            ao_clean,
            gi_clean
        )
        
        return final
    
    def rasterize_gbuffer(self, scene, camera, width, height):
        # Rasterize geometry to G-buffer
        return {'depth': None, 'normal': None, 'albedo': None}
    
    def trace_shadows(self, gbuffer, scene, samples_per_pixel):
        # Ray trace shadows
        return None
    
    def trace_reflections(self, gbuffer, scene, samples_per_pixel):
        # Ray trace reflections
        return None
    
    def trace_ambient_occlusion(self, gbuffer, scene, samples_per_pixel):
        # Ray trace AO
        return None
    
    def trace_global_illumination(self, gbuffer, scene, samples_per_pixel):
        # Ray trace 1-bounce GI
        return None
    
    def denoise(self, noisy, gbuffer):
        # Spatiotemporal denoising
        return noisy
    
    def compose(self, gbuffer, shadows, reflections, ao, gi):
        # Combine all layers
        return None

print("✓ Hybrid renderer concept loaded")

## Example: Temporal Accumulation Demonstration

In [ ]:
# Demonstrate temporal accumulation effect
noisy_single, accumulated = demonstrate_temporal_accumulation()

fig, axes = plt.subplots(1, 2, figsize=(12, 6))

axes[0].imshow(noisy_single)
axes[0].set_title('Single Frame (1 SPP, Noisy)')
axes[0].axis('off')

axes[1].imshow(accumulated)
axes[1].set_title('Temporal Accumulation (30 frames)')
axes[1].axis('off')

plt.tight_layout()
plt.show()

print("Temporal accumulation significantly reduces noise over time!")
print("This enables 1 SPP real-time ray tracing with acceptable quality.")

## Summary

**Advanced ray tracing** for real-time and production:

### Key Concepts

1. **Real-Time Ray Tracing**
   - 1-4 SPP per frame
   - 30-60 FPS target
   - Hybrid rasterization + RT
   - Hardware acceleration (RTX)

2. **Temporal Accumulation**
   - Blend with previous frames
   - Motion vector reprojection
   - Disocclusion detection
   - Exponential moving average

3. **Spatiotemporal Denoising (SVGF)**
   - Temporal accumulation first
   - Variance estimation
   - À-trous wavelet filter
   - Edge-stopping (depth, normal, luminance)
   - Variance-guided kernel size

4. **ReSTIR**
   - Weighted reservoir sampling
   - Temporal reuse
   - Spatial reuse
   - Handles thousands of lights

5. **Hybrid Rendering**
   - G-buffer from rasterization
   - Selective ray tracing
   - Per-effect denoising
   - Layer composition

### Modern Games Using RT

- **Metro Exodus**: Global illumination
- **Control**: Reflections, GI, contact shadows
- **Cyberpunk 2077**: Full RT lighting
- **Minecraft RTX**: Path tracing
- **Spider-Man**: Reflections

### Performance Techniques

✅ **1 SPP + denoising** instead of 100+ SPP  
✅ **Half-resolution RT** upscaled  
✅ **Temporal accumulation** for free samples  
✅ **Checkerboard rendering** alternate pixels  
✅ **Variable rate shading** less work in periphery  
✅ **ReSTIR** for many lights  

### Hardware Evolution

**RTX 2000 series** (2018):
- First RT cores
- ~30 FPS at 1080p

**RTX 3000 series** (2020):
- 2x faster RT
- 60 FPS at 1440p

**RTX 4000 series** (2022):
- 3rd gen RT cores
- DLSS 3 (frame generation)
- 60 FPS at 4K

### Future Directions

- **Neural rendering**: AI-based denoising (DLSS)
- **Hybrid path tracing**: Offline quality in real-time
- **Hardware ray tracing**: Dedicated silicon
- **Coherent ray tracing**: Better GPU utilization
- **Sparse ray tracing**: Adaptive sample distribution

Real-time ray tracing is now practical and becoming standard in modern games and professional applications!